### **Importing Libraries**




In [ ]:
import os
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive')

### **Gong Processing**

In [ ]:
def filter_gong_data(input_file, output_file, hosts):
    # Define the columns to extract from the data
    gong_columns = [
        'Gong Internal Company ID', 'Gong Internal Call ID', 'Gong Internal Owner ID', 'Call Title',
        'Recorded Date', 'Recorded Year', 'Recorded Month', 'Recorded Day of Month', 'Recorded Week',
        'Status', 'Conferencing Provider', 'Internal Meeting', 'Direction', 'Disposition', 'Duration (Sec.)',
        'Word Count', 'Host', 'Host Email Address', 'Host Title', 'Manager 1 Name', 'Manager 1 Email',
        'Manager 2 Name', 'Manager 2 Email', 'Manager 3 Name', 'Manager 3 Email', 'Manager 4 Name',
        'Manager 4 Email', 'Manager 5 Name', 'Manager 5 Email', 'Manager 6 Name', 'Manager 6 Email',
        'Manager 7 Name', 'Manager 7 Email', 'Manager 8 Name', 'Manager 8 Email', 'Manager 9 Name',
        'Manager 9 Email', 'Manager 10 Name', 'Manager 10 Email', 'CRM - Account 1 Name', 'CRM - Lead 1 Name'
    ]

    # Read the CSV file and select the relevant columns
    df = pd.read_csv(input_file, usecols=gong_columns)

    # Filter the data based on the specified hosts
    filtered_df = df[df['Host'].isin(hosts)]

    # Export the filtered DataFrame to CSV
    filtered_df.to_csv(output_file, sep=',', encoding='utf-8', index=False)


def main():
    # Specify the input and output file paths
    input_file = '/content/Extensive_Call_Data_3799966398572700455_Jun-14-2023_06-33-26_4669043425477227646.csv'
    output_file = 'Gong_Raw_Processed.csv'

    # Specify the hosts to filter the data
    hosts = [
        'Afan Haque', 'AJ Patwa', 'Chris Matthews', 'Christian Braswell', 'Cisilia Chan', 'Colin Chan', 'Emma Ewing',
        'Haaris Kirmani', 'Joel Cummings', 'Joel Farnsworth', 'Julia Blalock', 'Julia Pineda', 'Kathleen Hiemstra',
        'Stephanie Gamble', 'Talia Bond', 'Sibel Gilani', 'Ahmed Raza Chohan', 'Marco Rios', 'Jordan Broyles',
        'Katie Shank', 'Raahym Moeen', 'Ashraf Akhtar', 'Laraib Amjad', 'Anna Walsh', "Morena D'Alma"
    ]

    # Filter the Gong data based on hosts
    filter_gong_data(input_file, output_file, hosts)

    print(f"Filtered data exported to: {output_file}")


if __name__ == '__main__':
    main()


Filtered data exported to: Gong_Raw_Processed.csv


### **Intercom Processing**

In [ ]:
# Adding the relevant columns we need from the data
input_file = '/content/inbox-data-export.a540xj2u.a0773e50-68f7-4f77-9859-b7b8b3604b71.csv'
output_file = 'Intercom Processed.csv'
start_from = 119703

# Defining a function to process the data
def intercom_process(input_file, output_file, start_from=None):
    intercom_columns = [
        'Conversation ID', 'Conversation URL', 'Title', 'Conversation status',
        'Conversation tags', 'Created at', 'Last updated at', 'Inbound/Outbound',
        'Reopened', 'Closed', 'Conversation rating', 'Conversation rating requested',
        'Conversation rating remark', 'Email', 'Location', 'Name',
        'Assigned to (name)', 'Closed by (name)', 'Teammates participated',
        'Time to first reply (seconds)', 'Time to last close (seconds)',
        'Teammate replies', 'Channel'
    ]

    try:
        # Read the CSV file and select only the specified columns
        df_intercom = pd.read_csv(input_file, usecols=intercom_columns)

        # Filter the dataframe based on the start_from value
        if start_from is not None:
            df_intercom = df_intercom[df_intercom['Conversation ID'] >= start_from]

        # Sort the dataframe by column A in ascending order
        df_intercom.sort_values(by='Conversation ID', ascending=True, inplace=True)

        # Export the sorted dataframe to CSV
        df_intercom.to_csv(output_file, sep=',', encoding='utf-8', index=False)

        # Return the output file path
        return output_file
    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Call the function on the relevant data
processed_file = intercom_process(input_file, output_file, start_from)

# Print the processed file path
if processed_file:
    print("Data processed and saved to:", processed_file)



Data processed and saved to: Intercom Processed.csv


### **Historical Customer Raw Processing**

In [ ]:
def match_columns(csv_folder, predefined_columns):


    # Iterate over all CSV files in the folder
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            # Get the file path
            csv_file = os.path.join(csv_folder, file)

            # Read the CSV file
            df = pd.read_csv(csv_file)

            # Get the existing column names in the CSV file
            existing_columns = df.columns.tolist()

            # Identify the missing columns
            missing_columns = list(set(predefined_columns) - set(existing_columns))

            # Add the missing columns to the DataFrame
            for column in missing_columns:
                df[column] = None

            # Reorder the columns according to the predefined column order
            df = df[predefined_columns]

            # Save the modified DataFrame back to the CSV file
            df.to_csv(csv_file, index=False)


def process_csv_files(csv_folder, reset_columns):
    result_df = pd.DataFrame()

    # Iterate over all CSV files in the folder
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            # Get the file path
            csv_file = os.path.join(csv_folder, file)

            # Read the CSV file
            df = pd.read_csv(csv_file)

            # Apply data processing logic
            processed_data = df.loc[df['Date'] == df['Date'].min()]

            processed_data['Technician Hours logged against Services'] = processed_data['Technician Minutes logged against Services'] / 60
            processed_data['Technician Hours clocked into workday'] = processed_data['Technician Minutes clocked into workday'] / 60
            processed_data['Internal Review Request Sent'] = processed_data['Internal Review Requests Via Email'] + processed_data['Internal Review Requests Via SMS']
            processed_data['Total payment Receipts Sent to customer'] = processed_data['Payment Receipts Sent Via Email'] + processed_data['Payment Receipts Sent Via SMS']
            processed_data['Total Estimates sent to customers'] = processed_data['Estimates Sent Via Email'] + processed_data['Estimates Sent Via SMS']
            processed_data['Total Invoices sent to customers'] = processed_data['Invoices Sent Via Email'] + processed_data['Invoices Sent Via SMS']
            processed_data['Motor Parts + Labour items'] = processed_data['MOTOR Parts Added to Services'] + processed_data['MOTOR Labor Items Added to Services']

            # Reset values to 0 if greater than 100 for specified columns
            for column in reset_columns:
                processed_data.loc[processed_data[column] > 100, column] = 0

            # Append processed data to the result DataFrame
            result_df = pd.concat([result_df, processed_data], ignore_index=True)

    return result_df


# Set the folder path containing the CSV files
csv_folder = '/content/drive/MyDrive/Your_CSV_Folder'

# Specify the predefined column names
predefined_columns = ['Date',	'Salesforce ID',	'Name',	'Customers Created',	'New Email Recorded',	'Vehicle Created',	'New ROs Created',	'Services Created',	'Line Items Created',	'Revenue',	'Payment Taken',	'Appointment Created',
               'Appointment Created with Customer associated',	'Appointment Created with Vehicle associated',	'Appointment Created with RO associated',	'Appointment Reminders Sent',	'Appointment Confirmations Received',
               'Estimates Sent Via Email',	'Estimates Sent Via SMS',	'Estimates Approved',	'Invoices Sent Via Email',	'Invoices Sent Via SMS',	'Payment Reciepts Sent Via Email',	'Payment Reciepts Sent Via SMS',
               'Internal Review Requests Via Email',	'Internal Review Requests Via SMS',	'Internal Reviews Received',	'Internal Reviews at or above Minimum Rating Received',	'Google Review Links Clicked',
               'New Google Reviews Received',	'Invoice Posted to QBO',	'MOTOR Canned Services Added to ROs',	'MOTOR Parts Added to Services',	'MOTOR Labor Items Added to Services',	'Vehicle Linked to MOTOR',
               'Canned Service Added to ROs',	'Inspection Checklists Associated to Services',	'Technician Minutes clocked into workday',	'Technician Minutes logged against Services',	'Services Completed by Technicians',
               'PartsTech Parts Orders Created',	'PartsTech Parts Orders Submitted',	'PartsTech Parts used on Services']

# Specify the columns for resetting values to 0 if > 100
reset_columns = [
    'Customers Created',
    'New Email Recorded',
    'Vehicle Created',
    'New ROs Created',
    'Services Created',
    'Line Items Created',
]

# Step 1: Match columns in all CSV files
match_columns(csv_folder, predefined_columns)

# Step 2: Process the CSV files and obtain consolidated data
processed_data = process_csv_files(csv_folder, reset_columns)

# Save the consolidated data to a CSV file
processed_data.to_csv('/content/drive/MyDrive/Consolidated_Data.csv', index=False)


### **Customer Raw Processing**


In [ ]:
# Call the function with the updated input and output file names
input_cust_raw = '/content/Alp-Health-Report-2023-06-04to2023-06-03.csv'
output_cust_raw = 'Customer Raw Processed.csv'

def process_customer_data(input_cust_raw, output_cust_raw):
    # Define the columns needed from the data
    raw_columns = [
        'Date', 'Salesforce ID', 'Name', 'Customers Created', 'New Email Recorded',
        'Vehicle Created', 'New ROs Created', 'Services Created', 'Line Items Created',
        'Revenue', 'Payment Taken', 'Appointment Created',
        'Appointment Created with Customer associated',
        'Appointment Created with Vehicle associated',
        'Appointment Created with RO associated', 'Appointment Reminders Sent',
        'Appointment Confirmations Received', 'Estimates Sent Via Email',
        'Estimates Sent Via SMS', 'Estimates Approved', 'Invoices Sent Via Email',
        'Invoices Sent Via SMS', 'Payment Reciepts Sent Via Email',
        'Payment Reciepts Sent Via SMS', 'Internal Review Requests Via Email',
        'Internal Review Requests Via SMS', 'Internal Reviews Received',
        'Internal Reviews at or above Minimum Rating Received',
        'Google Review Links Clicked', 'New Google Reviews Received',
        'Invoice Posted to QBO', 'MOTOR Canned Services Added to ROs',
        'MOTOR Parts Added to Services', 'MOTOR Labor Items Added to Services',
        'Vehicle Linked to MOTOR', 'Canned Service Added to ROs',
        'Inspection Checklists Associated to Services',
        'Technician Minutes clocked into workday',
        'Technician Minutes logged against Services',
        'Services Completed by Technicians', 'PartsTech Parts Orders Created',
        'PartsTech Parts Orders Submitted', 'PartsTech Parts used on Services'
    ]

    # Read the CSV file and select only the specified columns
    df = pd.read_csv(input_cust_raw, usecols=raw_columns)

    # Filter the dataframe to get the row with the minimum date
    min_date = df['Date'].min()
    df_filtered = df[df['Date'] == min_date].copy()

    # Calculate additional columns
    df_filtered['Technician Hours logged against Services'] = df_filtered['Technician Minutes clocked into workday'] / 60
    df_filtered['Technician Hours clocked into workday'] = df_filtered['Inspection Checklists Associated to Services'] / 60
    df_filtered['Internal Review Request Sent'] = df_filtered['Internal Review Requests Via Email'] + df_filtered['Internal Review Requests Via SMS']
    df_filtered['Total payment Reciepts Sent to customer'] = df_filtered['Payment Reciepts Sent Via Email'] + df_filtered['Payment Reciepts Sent Via SMS']
    df_filtered['Total Estimates sent to customers'] = df_filtered['Estimates Sent Via Email'] + df_filtered['Estimates Sent Via SMS']
    df_filtered['Total Invoices sent to customers'] = df_filtered['Invoices Sent Via Email'] + df_filtered['Invoices Sent Via SMS']
    df_filtered['Motor Parts + Labour items'] = df_filtered['MOTOR Parts Added to Services'] + df_filtered['MOTOR Labor Items Added to Services']

    # Export the processed dataframe to CSV
    df_filtered.to_csv(output_cust_raw, sep=',', encoding='utf-8', index=False)

    # Return the output file path
    return output_cust_raw

processed_file = process_customer_data(input_cust_raw, output_cust_raw)

# Print the processed file path
if processed_file:
    print("Data processed and saved to:", processed_file)


Data processed and saved to: Customer Raw Processed.csv
